In [171]:
from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec
import json, yaml

In [172]:
with open("openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_=data, max_value_length=4000)

In [173]:
def get_api_operation_by_id(json_spec, operation_id):
    paths = json_spec.dict_.get("paths", {})
    for path, methods in paths.items():
        for method, operation in methods.items():
            # Check if 'operation' is a dictionary
            if isinstance(operation, dict):
                operation_id = operation.get("operationId")
                
                if operation_id == operation_id:
                    return operation, method
            else:
                # Handle the case where 'operation' is not a dictionary
                print(f"Skipping invalid operation: {operation}")


In [174]:
import re

def resolve_refs(input_dict, json_spec):
    # Check if the input_dict is a dictionary and contains a '$ref' key
    if isinstance(input_dict, dict) and '$ref' in input_dict:
        ref_value = input_dict['$ref']
        
        # Use regular expression to extract the schema name
        match = re.match(r'#/components/schemas/(\w+)', ref_value)
        if match:
            schema_name = match.group(1)
            
            # Try to find the corresponding schema in the json_spec dictionary
            schema = json_spec.dict_.get("components", {}).get("schemas", {}).get(schema_name)
            
            # If a matching schema is found, replace the '$ref' key with the schema
            if schema:
                return schema
        
    # Recursively process nested dictionaries and lists
    if isinstance(input_dict, dict):
        for key, value in input_dict.items():
            input_dict[key] = resolve_refs(value, json_spec)
    elif isinstance(input_dict, list):
        for i, item in enumerate(input_dict):
            input_dict[i] = resolve_refs(item, json_spec)
    
    return input_dict

import re

# ref_list -> [{'$ref': '#/components/parameters/QueryAlbumIds'},
#  {'$ref': '#/components/parameters/QueryMarket'}]

def hydrateParams(json_spec, ref_list):
    last_portion_list = []
    
    for ref in ref_list:
        match = re.search(r'/([^/]+)$', ref['$ref'])
        if match:
            last_portion_list.append(json_spec["components"]["parameters"][match.group(1)])
    
    return last_portion_list

In [175]:
def process_api_operation(method, api_operation, json_spec):
    if method == "get":
        return api_operation
    else:
        request_schema = api_operation["requestBody"]["content"]["application/json"]["schema"]
        resolved_request = resolve_refs(request_schema, json_spec.dict_)
        return resolved_request

In [176]:
api_operation, method=get_api_operation_by_id(json_spec, "get-information-about-the-users-current-playback")
isolated_request = process_api_operation(method, api_operation, json_spec)
isolated_request["parameters"] = hydrateParams(json_spec.dict_, isolated_request["parameters"])

isolated_request

{'description': 'Get Spotify catalog information for multiple albums identified by their Spotify IDs.\n',
 'operationId': 'get-multiple-albums',
 'parameters': [{'in': 'query',
   'name': 'ids',
   'required': True,
   'schema': {'description': 'A comma-separated list of the [Spotify IDs](/documentation/web-api/concepts/spotify-uris-ids) for the albums. Maximum: 20 IDs.\n',
    'example': '382ObEPsp2rxGrnsizN5TX,1A2GTWGtFfWp7KSQTwWOyo,2noRn2Aes5aoNVsU6iWThc',
    'title': 'Spotify Album IDs',
    'type': 'string'}},
  {'in': 'query',
   'name': 'market',
   'required': False,
   'schema': {'description': 'An [ISO 3166-1 alpha-2 country code](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2).\n  If a country code is specified, only content that is available in that market will be returned.<br/>\n  If a valid user access token is specified in the request header, the country associated with\n  the user account will take priority over this parameter.<br/>\n  _**Note**: If neither market or